In [1]:
import tensorflow as tf
# import cv2
import tensorflow.keras as K
from keras import Model
from keras.layers import Lambda
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,LeakyReLU,BatchNormalization,Activation, MaxPooling2D, Dropout, Dense, Flatten, \
    GlobalAveragePooling2D,  SeparableConv2D, GlobalMaxPooling2D,Input,Layer,add,concatenate,UpSampling2D,dot


In [6]:
def cbl_cell(img_input,filters,kernels,stride=(1,1)):
    origin = Conv2D(filters, kernels,strides =stride,use_bias=False,padding='SAME')(img_input)
    x = BatchNormalization()(origin)
    x = LeakyReLU(alpha=0.05)(x)
    return x

In [3]:
img_input = Input(shape=(441,399,1))
x = cbl_cell(img_input)
x

Metal device set to: Apple M1


2021-11-16 22:15:10.336586: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-16 22:15:10.336844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<KerasTensor: shape=(None, 441, 399, 32) dtype=float32 (created by layer 'leaky_re_lu')>

In [3]:
def res1(inputs,filter_num,stride=(1,1)):
    x = cbl_cell(inputs,filter_num,stride)
    x = cbl_cell(inputs,2*filter_num,stride)
    x = add([inputs,x])
    return x
    

In [4]:
def static_res_block(inputs,filters,n):
    x = res1(inputs, filters)
    for i in range(n - 1):
        x = res1(x, filters)
    return x

In [19]:
x = static_res_block(x,32,2)

In [7]:
img_input = Input(shape=(441,399,1))
x = cbl_cell(img_input,32,(3,3))
x = cbl_cell(x,64,(3,3),(2,2))
x = static_res_block(x,32,1)
x = cbl_cell(x,128,(3,3),(2,2))
x = static_res_block(x,64,2)
x = cbl_cell(x,256,(3,3),(2,2))
x = static_res_block(x,128,8)
x = cbl_cell(x,512,(3,3),(2,2))
x = static_res_block(x,256,8)
x = cbl_cell(x,1024,(3,3),(2,2))
x = static_res_block(x,512,4)
x

<KerasTensor: shape=(None, 14, 13, 1024) dtype=float32 (created by layer 'add_42')>

In [13]:
class GAN_Network():
    def __init__(self):
        self.w_init = K.initializers.random_normal(stddev=0.02)
        self.g_init = K.initializers.random_normal(1.,0.02)

    def GAN_G(self):
        nin = Input(shape=(56,50,576))
        resize = K.layers.Reshape((64,36,-1))(nin)
        n = Conv2D(64,3,1,activation='relu',padding='SAME')(resize)
        temp = n
        for i in range(16):
            nn = Conv2D(64, (3, 3), (1, 1), padding='SAME')(n)
            nn = BatchNormalization()(nn)
            nn = Conv2D(64, (3, 3), (1, 1), padding='SAME')(nn)
            nn = BatchNormalization()(nn)
            nn = K.layers.multiply([n,nn])
            # nn = Lambda(lambda x: K.backend.batch_dot(*x))([n,nn])
            n = nn
        n = Conv2D(64, (3, 3), (1, 1), padding='SAME')(n)
        n = BatchNormalization()(n)
        nn = K.layers.multiply([n,temp])
        # nn = add([n, temp])
        n = Conv2D(256, (3, 3), (1, 1), padding='SAME')(nn)
        n = UpSampling2D(2)(n)

        n = Conv2D(256, (3, 3), (1, 1), activation='relu', padding='SAME')(n)
        n = UpSampling2D(2)(n)

        nn = Conv2D(3, (1, 1), (1, 1), activation='relu', padding='SAME')(n)
        M = Model(nin,nn)
        return M

In [14]:
x = GAN_Network()
y = x.GAN_G()
y.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 56, 50, 576) 0                                            
__________________________________________________________________________________________________
reshape_6 (Reshape)             (None, 64, 36, 700)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_74 (Conv2D)              (None, 64, 36, 64)   403264      reshape_6[0][0]                  
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 64, 36, 64)   36928       conv2d_74[0][0]                  
____________________________________________________________________________________________

In [6]:
nin = Input(shape=(56,50,576))

In [8]:
nin = K.layers.Reshape((256,144,45))(nin)

ValueError: total size of new array must be unchanged, input_shape = [56, 50, 576], output_shape = [256, 144, 45]

In [56]:
nin.shape

TensorShape([None, 64, 36, 700])